In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from data_input import LoadData
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
data_dir = '/mnt/c/senior_project/data'
df_arr = []
control_arr = []
session_folders = []
skilled_session = ['session_2', 'session_3', 'session_4', 'session_8', 'session_9', 'session_11']
session_ids = [10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# NOTE: file structure:
#   -/ data
#       -/control
#           -/ session_#
#               - files
#       -/participants
#           -/ session_#
#               - files
session_folders = []
for _, folders, _ in os.walk(data_dir):
    for player_type in folders:
        player_dir = os.path.join(data_dir, player_type)
        for path, folders, files in os.walk(player_dir):
            for folder in folders:
                folder_dir = os.path.join(player_dir, folder) 
                session_folders.append(folder)
                for path, folders, files in os.walk(folder_dir):
                    hr_file = os.path.join(folder_dir, files[2])
                    timestamp_file = os.path.join(folder_dir, files[3])
                    tracking_file = os.path.join(folder_dir, files[4])
                    if folder in skilled_session or player_type == 'control':
                        skilled_flag = True
                    else:
                        skilled_flag = False
                    if player_type == 'control':
                        if folder == 'session_13':
                            control_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=-21.53611).dataframe)
                        else:
                            control_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=7).dataframe)
                    else:
                        df_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=7).dataframe)
print(session_folders)

Verify Dataframe Creation

In [ ]:
i = 0
for df in control_arr:
    print(f"Session {session_ids[i]} Dataframe")
    i += 1
    print(df)

In [ ]:
i = 0
for df in df_arr:
    print(f"{session_folders[i]} Dataframe")
    i += 1
    print(df)

All participant HR 

In [ ]:
# Initial plots of hr during the sessions with skill labels

color_map = {1: 'blue', 0: 'red'}
label_map = {1: 'skilled', 0: 'unskilled'}

# Plot the data
for df in df_arr:
    plt.plot(df['Interval'], df['HR'], label=label_map[df['Label'].iloc[0]], color=color_map[df['Label'].iloc[0]])

plt.xlabel('Interval')
plt.ylabel('Heart Rate (HR)')
plt.title('Heart Rate Over Time by Session')
plt.legend()
plt.grid(True)
plt.show()

Check HR variance during session

In [ ]:
# Bar graph of min HR and max heart of each participant during session

sessions = [i+2 for i in range(len(df_arr))]

# Plot the data
diff = []
labels = []
colors = []
for df in df_arr:
    max_hr = df['HR'].max()
    min_hr = df['HR'].min()
    diff.append(max_hr - min_hr)
    colors.append(color_map[df['Label'].iloc[0]])
    labels.append(label_map[df['Label'].iloc[0]])
#     i += 1
#     plt.plot(df['Interval'], df['HR'], label=df['Label'].iloc[0], color=color_map[df['Label'].iloc[0]])
plt.bar(sessions, diff, label=labels, color=colors)
plt.xlabel('Session')
plt.ylabel('Heart Rate (HR)')
plt.title('Heart Rate Difference By Session')
plt.legend()
plt.grid(True)
plt.show()

All Participant APM 

In [ ]:
# Initial APM graphs

color_map = {1: 'blue', 0: 'red'}
label_map = {1: 'skilled', 0: 'unskilled'}

# Plot the data
# plt.figure(figsize=(12, 8))

for df in df_arr:
    plt.plot(df['Interval'], df['APM'], label=label_map[df['Label'].iloc[0]], color=color_map[df['Label'].iloc[0]])

plt.xlabel('Interval')
plt.ylabel('APM')
plt.title('APM Over Time by Session')
plt.legend()
plt.grid(True)
plt.show()

Control Player HR variance

In [ ]:
sessions = [i+2 for i in range(len(control_arr))]

diff = []
labels = []
colors = []
for df in control_arr:
    max_hr = df['HR'].max()
    min_hr = df['HR'].min()
    diff.append(max_hr - min_hr)

plt.bar(sessions, diff)
plt.xlabel('Session')
plt.ylabel('Heart Rate (HR)')
plt.title('Heart Rate Difference By Session')
plt.legend()
plt.grid(True)
plt.show()

Control Player APM by Session

In [ ]:
# Initial APM graphs

color_map = {1: 'blue', 0: 'red'}

i = 2
for df in control_arr:
    plt.plot(df['Interval'], df['APM'], label=f'session {i}')
    i += 1

plt.xlabel('Interval')
plt.ylabel('APM')
plt.title('APM Over Time by Session')
plt.legend()
plt.grid(True)
plt.show()

Control Vs Participant APM by Session

In [ ]:
importance_map = {0: 'blue', 1: 'red', 2: 'green'}

for session in range(len(df_arr)):

    fig = go.Figure()
    control = control_arr[session]
    control['Group'] = 'Control'
    p_df = df_arr[session]
    p_df['Group'] = 'Participant'
    df = pd.concat([control, p_df])
    for group, color, dash in [('Control', 'blue', 'solid'), 
                            ('Participant', 'red', 'dash')]:
        group_df = df[df['Group'] == group]
        fig.add_trace(go.Scatter(
            x=group_df['Interval'],
            y=group_df['APM'],
            mode='lines',
            name=group,
            line=dict(color=color, dash=dash, width=2),
        ))

    importance_colors = {
        0: ('Low', 'AliceBlue'),
        1: ('Medium', 'MediumSeaGreen'),
        2: ('High', 'LightCoral')
    }

    for imp, (label, color) in importance_colors.items():
        fig.add_trace(go.Scatter(
            x=[None], 
            y=[None],
            mode='lines',
            name=label,
            showlegend=True
        ))

    # find when event happens (i.e low to medium)
    df['importance_change'] = df['Importance'].ne(df['Importance'].shift())
    importance_intervals = df[df['importance_change']]

    for i in range(len(importance_intervals)):
        start = importance_intervals.iloc[i]['Interval']
        end = df['Interval'].max() if i == len(importance_intervals)-1 else importance_intervals.iloc[i+1]['Interval']
        imp = importance_intervals.iloc[i]['Importance']
        
        fig.add_vrect(
            x0=start, 
            x1=end,
            fillcolor=importance_colors[imp][1],
            layer="below",
        )

    fig.update_layout(
        title=f'APM Comparison of Session {session_ids[session]}',
        xaxis_title='Interval (10 Seconds)',
        yaxis_title='APM (Actions Per Minute)',
        hovermode='x unified',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
    )

    fig.show()

Control Vs Participant HR with Base HR by Session

In [ ]:
importance_map = {0: 'blue', 1: 'red', 2: 'green'}

for session in range(len(df_arr)):
    # NOTE: session 7 had EmotiBit issues
    if session == 7:
        pass
    else:
        fig = go.Figure()
        control = control_arr[session]
        control['Group'] = 'Control'
        p_df = df_arr[session]
        p_df['Group'] = 'Participant'
        df = pd.concat([control, p_df])
        
        for group, color, dash in [('Control', 'blue', 'solid'), 
                                ('Participant', 'red', 'solid')]:
            group_df = df[df['Group'] == group]
            # plot Base HR accross plot
            fig.add_trace(go.Scatter(
                x=[group_df['Interval'].min(), group_df['Interval'].max()],  # Span all intervals
                y=[group_df['Base HR'].iloc[0], group_df['Base HR'].iloc[0]],  # Constant base HR
                mode='lines',
                name=f'{group} Base HR',
                line=dict(color=color, width=2, dash='dot'),
                showlegend=True
            ))

            fig.add_trace(go.Scatter(
                x=group_df['Interval'],
                y=group_df['HR'],
                mode='lines',
                name=group,
                line=dict(color=color, dash=dash, width=2),
            ))

        importance_colors = {
            0: ('Low', 'AliceBlue'),
            1: ('Medium', 'MediumSeaGreen'),
            2: ('High', 'LightCoral')
        }

        for imp, (label, color) in importance_colors.items():
            fig.add_trace(go.Scatter(
                x=[None], 
                y=[None],
                mode='lines',
                name=label,
                showlegend=True
            ))

        df['importance_change'] = df['Importance'].ne(df['Importance'].shift())
        importance_intervals = df[df['importance_change']]

        for i in range(len(importance_intervals)):
            start = importance_intervals.iloc[i]['Interval']
            end = df['Interval'].max() if i == len(importance_intervals)-1 else importance_intervals.iloc[i+1]['Interval']
            imp = importance_intervals.iloc[i]['Importance']
            
            fig.add_vrect(
                x0=start, 
                x1=end,
                fillcolor=importance_colors[imp][1],
                layer="below",
            )

        fig.update_layout(
            title=f'HR Comparison of Session {session_ids[session]}',
            xaxis_title='Interval (10 Seconds)',
            yaxis_title='Heart Rate (Beats Per Minute)',
            hovermode='x unified',
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=1.02,
                xanchor='right',
                x=1
            ),
        )

        fig.show()

Average By Intervals and By Event Type

In [ ]:
p_dataframe = pd.concat(df_arr)
c_dataframe = pd.concat(control_arr)

control_hr_mean_interval = c_dataframe.groupby('Interval')['HR'].mean().reset_index()
control_hr_mean_interval['Type'] = 'Control'
unskilled_hr_mean_interval = p_dataframe.loc[p_dataframe['Label'] == 0].groupby('Interval')['HR'].mean().reset_index()
unskilled_hr_mean_interval['Type'] = 'Unskilled'
skilled_hr_mean_interval = p_dataframe[p_dataframe['Label'] == 1].groupby('Interval')['HR'].mean().reset_index()
skilled_hr_mean_interval['Type'] = 'Skilled'

hr_mean_interval = pd.concat([control_hr_mean_interval, unskilled_hr_mean_interval, skilled_hr_mean_interval], axis=0)

# Create graph
fig = px.line(hr_mean_interval, x='Interval', y='HR', color='Type', title='Mean Heart Rate by Group')
fig.update_layout(
    xaxis_title='Interval (10 Seconds)',
    yaxis_title='Heart Rate (Beats Per Minute)',
    hovermode='x unified',
)
fig.show()


In [ ]:
p_dataframe = pd.concat(df_arr)
c_dataframe = pd.concat(control_arr)

control_APM_mean_interval = c_dataframe.groupby('Interval')['APM'].mean().reset_index()
control_APM_mean_interval['Type'] = 'Control'
unskilled_APM_mean_interval = p_dataframe.loc[p_dataframe['Label'] == 0].groupby('Interval')['APM'].mean().reset_index()
unskilled_APM_mean_interval['Type'] = 'Unskilled'
skilled_APM_mean_interval = p_dataframe[p_dataframe['Label'] == 1].groupby('Interval')['APM'].mean().reset_index()
skilled_APM_mean_interval['Type'] = 'Skilled'

APM_mean_interval = pd.concat([control_APM_mean_interval, unskilled_APM_mean_interval, skilled_APM_mean_interval], axis=0)

fig = px.line(APM_mean_interval, x='Interval', y='APM', color='Type', title='Mean APM by Group')
fig.update_layout(
    xaxis_title='Interval (10 Seconds)',
    yaxis_title='APM (Actions Per Minute)',
    hovermode='x unified',
)
fig.show()

In [ ]:
p_dataframe = pd.concat(df_arr)
c_dataframe = pd.concat(control_arr)

# create 3 dataframes (1 for each player type)
control_hr_mean_Importance = c_dataframe.groupby('Importance')['HR'].mean().reset_index()
control_hr_mean_Importance['Type'] = 'Control'
unskilled_hr_mean_Importance = p_dataframe.loc[p_dataframe['Label'] == 0].groupby('Importance')['HR'].mean().reset_index()
unskilled_hr_mean_Importance['Type'] = 'Unskilled'
skilled_hr_mean_Importance = p_dataframe[p_dataframe['Label'] == 1].groupby('Importance')['HR'].mean().reset_index()
skilled_hr_mean_Importance['Type'] = 'Skilled'

hr_mean_Importance = pd.concat([control_hr_mean_Importance, unskilled_hr_mean_Importance, skilled_hr_mean_Importance], axis=0)

fig = px.line(hr_mean_Importance, x='Importance', y='HR', color='Type', title='Mean Heart Rate During Event by Group')
fig.update_layout(
    xaxis_title='Interval (10 Seconds)',
    yaxis_title='Heart Rate (Beats Per Minute)',
    hovermode='x unified',
)
fig.show()

In [ ]:
p_dataframe = pd.concat(df_arr)
c_dataframe = pd.concat(control_arr)

control_APM_mean_Importance = c_dataframe.groupby('Importance')['APM'].mean().reset_index()
control_APM_mean_Importance['Type'] = 'Control'
unskilled_APM_mean_Importance = p_dataframe.loc[p_dataframe['Label'] == 0].groupby('Importance')['APM'].mean().reset_index()
unskilled_APM_mean_Importance['Type'] = 'Unskilled'
skilled_APM_mean_Importance = p_dataframe[p_dataframe['Label'] == 1].groupby('Importance')['APM'].mean().reset_index()
skilled_APM_mean_Importance['Type'] = 'Skilled'

APM_mean_Importance = pd.concat([control_APM_mean_Importance, unskilled_APM_mean_Importance, skilled_APM_mean_Importance], axis=0)

fig = px.line(APM_mean_Importance, x='Importance', y='APM', color='Type', title='Mean APM During Event by Group')
fig.update_layout(
    xaxis_title='Interval (10 Seconds)',
    yaxis_title='APM (Actions Per Minute)',
    hovermode='x unified',
)
fig.show()

Explore Features For Models

In [ ]:
dataframe = pd.concat(df_arr)
dataframe = dataframe.drop(columns='Interval')

features = ['HR', 'APM', 'Importance', 'Base HR']

fig = px.scatter_matrix(
    dataframe, 
    dimensions=features,
    color='Label'
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
fig = px.parallel_coordinates(dataframe, color='Label', labels={'Label': 'Skill Level',
                                                                'HR': 'Heart Rate', 'APM': 'APM', 'Base HR': 'Base HR', 'Importance': 'Event Level'})

fig.show()

Control Player Data (May not use in Models)

In [ ]:
dataframe = pd.concat(control_arr)
dataframe = dataframe.drop(columns='Interval')

features = ['HR', 'APM', 'Importance', 'Base HR']

fig = px.scatter_matrix(
    dataframe, 
    dimensions=features,
    color='Label'
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
fig = px.parallel_coordinates(dataframe, color='Label', labels={'Label': 'Skill Level',
                                                                'HR': 'Heart Rate', 'APM': 'APM', 'Base HR': 'Base HR', 'Importance': 'Event Level'})

fig.show()